In [1]:
import pandas as pd
from utils import *

In [2]:
df = pd.read_csv('results/tabular_metrics_results.csv')

In [3]:
df.score=df.apply(lambda x: x.score * 100 if x.metric in ["efficacy_test", "histogram_intersection"] else x.score, axis=1)
df=df.query("fake_size == 30932 or fake_size==103904 or fake_size==16858  or fake_size==1000 or fake_size==60127")
df[["dataset", "setting", "llm", "finetune_type"]] = df.apply(lambda x: get_dslf(x), axis=1, result_type="expand")
df["unique_name"] = df.apply(lambda x: "_".join(x['fake_path'].split("/")[1:-2]),axis=1)


df['bs']=df.fake_path.apply(lambda x: get_bs(x))
df['exp']=df.fake_path.apply(lambda x: get_exp(x))
df['folder_name']=df.fake_path.apply(lambda x: get_folder_name(x))
df['train_epoch']=df.fake_path.apply(lambda x: get_train_ep(x))

df['chkt_epoch'] = df.fake_path.apply(lambda x: get_epoch(x))
df['eps'] = df.fake_path.apply(lambda x: get_eps(x))
df['clip']=df.fake_path.apply(lambda x: get_clip(x))
df=df.fillna("None")

df["merged_metric_name"]=df.apply(lambda x: merge_metric_name(x), axis=1)
df["merged_metric_name2"]=df.apply(lambda x: merge_metric_name2(x), axis=1)

In [4]:
index=["exp", "dataset", "llm", "finetune_type", "folder_name", "bs", "train_epoch", "chkt_epoch","eps", "clip",'fake_size']

df['LR']=df.exp.apply(lambda x: x.split("-")[0].strip("LR") if "LR" in x else -1)
df['shuffle']=df.exp.apply(lambda x: x.split("_")[3].split("-")[-1] if "shuffle" in x else "-1")
df['wl']=df.exp.apply(lambda x: x.split("_")[-1].strip("wl") if "wl" in x else "-1")
df['impute']=df.folder_name.apply(lambda x: x.split("_")[-1].split("-")[-1] if "impute" in x else -1)
index=index+["LR", "shuffle", "impute", "wl"]

def f(x):
    
    if "ood-using-airline" in x:
        # print(x)
        return "ood-using-airline"
    elif "ood-using-adult" in x:
        return "ood-using-adult"
    elif "ood-using-texas" in x:
        return "ood-using-texas"
    elif "uniform" in x:
        
        return "uniform"
    elif "Standard" in x:
        return "standard"
    else:
        return -1
    
df['approach']=df.exp.apply(lambda x: f(x))
index=index+["approach"]

# Table 2

In [5]:
eps=1
query1=f"llm in ('ctgan', 'tvae', 'vae') and dataset in ('adult', 'airline', 'texas')"
query2=f"llm=='GPT2' and dataset in ('adult', 'airline', 'texas') and LR=='0.0005' and shuffle == 'True' and impute=='False' and eps=='nondp'"
query3=f"llm=='real' and dataset in ('adult', 'airline', 'texas')"


index=["dataset", "llm", "finetune_type", "folder_name", "bs", "train_epoch", "chkt_epoch","eps", "clip",'fake_size']
index=index+["LR", "shuffle", "impute", "wl"]
index=index+["approach"]
df_tmp=df.query(f"{query1} or {query2} or {query3}")
df_tmp=df_tmp[df_tmp['exp'] != 'baselines_adult']
df_tmp=df_tmp[df_tmp['exp'] != 'baseline-test_airline']
df_tmp = df_tmp[df_tmp.fake_path.apply(lambda x: 'baselines' not in x)]
df_tmp['score']=df_tmp.apply(lambda x: x['score'] * 100 if x['metric'] == 'correlation_accuracy' or x['metric'] == 'pairwise_similarity' else x['score'], axis=1)

columns=["merged_metric_name2"]
tabular_metrics=['efficacy_f1', 'efficacy_auc', 'efficacy_accuracy', 'correlation_accuracy', 'pairwise_mean', 'histogram_mean']
# assert len(df_tmp.fake_path.unique()) == (6*2*4) + (5*1*4)

ROUND=1
df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score")
df_tmpp = df_tmpp[~df_tmpp.efficacy_f1.isna()]
df_tmpp=df_tmpp.reset_index().set_index(['dataset', 'llm'])[tabular_metrics].loc[['adult', 'airline', 'texas'], ['real', 'ctgan', 'tvae', 'vae', 'GPT2'], :]

df_tmpp

/var/folders/z5/cs_w84312s1cxj7105940jzm0000gn/T/ipykernel_56685/1377558198.py:21: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score")


merged_metric_name2    efficacy_f1  efficacy_auc efficacy_accuracy  \
dataset llm                                                          
adult   real          69.9$\pm$2.0  91.7$\pm$1.0      84.0$\pm$4.0   
        ctgan         59.5$\pm$6.0  88.5$\pm$0.0      80.2$\pm$3.0   
        tvae          63.2$\pm$2.0  87.5$\pm$1.0      77.7$\pm$4.0   
        vae          53.8$\pm$10.0  86.6$\pm$1.0      80.5$\pm$1.0   
        GPT2          68.9$\pm$0.0  90.7$\pm$0.0      83.7$\pm$2.0   
airline real          90.6$\pm$8.0  96.2$\pm$5.0      91.8$\pm$7.0   
        ctgan         87.2$\pm$3.0  94.7$\pm$2.0      88.9$\pm$3.0   
        tvae          85.8$\pm$5.0  93.0$\pm$6.0      87.2$\pm$6.0   
        vae           79.8$\pm$1.0  91.1$\pm$1.0      80.0$\pm$1.0   
        GPT2          89.6$\pm$5.0  95.9$\pm$3.0      91.4$\pm$4.0   
texas   real          86.6$\pm$2.0  98.8$\pm$0.0      95.0$\pm$1.0   
        ctgan         80.8$\pm$4.0  96.8$\pm$2.0      92.8$\pm$2.0   
        tvae          76.7$\pm$7.0  95.3$\pm$3.0      90.9$\pm$3.0   
        vae           74.3$\pm$3.0  95.6$\pm$1.0      90.6$\pm$2.0   
        GPT2          83.9$\pm$2.0  98.6$\pm$0.0      93.6$\pm$1.0   

merged_metric_name2 correlation_accuracy pairwise_mean histogram_mean  
dataset llm                                                            
adult   real                97.3$\pm$nan  97.5$\pm$0.0   99.1$\pm$0.0  
        ctgan               74.9$\pm$4.0  85.0$\pm$1.0   91.2$\pm$1.0  
        tvae                76.6$\pm$2.0  84.5$\pm$1.0   91.5$\pm$1.0  
        vae                 65.3$\pm$2.0  60.2$\pm$2.0   73.3$\pm$3.0  
        GPT2                79.9$\pm$1.0  83.7$\pm$0.0   90.7$\pm$1.0  
airline real                97.2$\pm$nan  98.4$\pm$0.0   99.4$\pm$0.0  
        ctgan               84.4$\pm$1.0  89.3$\pm$1.0   94.4$\pm$1.0  
        tvae                78.0$\pm$3.0  82.8$\pm$3.0   90.3$\pm$2.0  
        vae                 67.8$\pm$5.0  60.4$\pm$1.0   76.8$\pm$0.0  
        GPT2                86.5$\pm$4.0  85.5$\pm$2.0   90.8$\pm$1.0  
texas   real                96.3$\pm$nan  98.9$\pm$0.0   99.5$\pm$0.0  
        ctgan               72.8$\pm$4.0  88.3$\pm$1.0   93.1$\pm$0.0  
        tvae                61.2$\pm$4.0  63.4$\pm$7.0   93.5$\pm$1.0  
        vae                 51.5$\pm$7.0  57.5$\pm$1.0   89.3$\pm$1.0  
        GPT2                72.8$\pm$1.0  82.8$\pm$3.0   94.9$\pm$0.0

# Table 3 and Appendix F.2, Table 8

The correlation Accuracy score for DP-2Stage-U, texas dataset doesn't exactly match the paper. Table below shows that it performs better than what was reported.  Paper reports $80.7\pm6$ but the table below shows $81.2\pm6$. I might have made a mistake when copying over the numbers.

In [6]:
eps=1
query1=f"llm in ('aim', 'mst', 'dpgan', 'dpctgan', 'dpvae') and dataset in('adult', 'airline', 'texas') and eps=={eps}"
query2=f"llm=='GPT2'and dataset in ('adult', 'airline', 'texas') and LR=='0.0005' and approach == 'standard' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '-1'"
query3=f"llm=='GPT2' and dataset in ('adult', 'airline', 'texas') and LR=='0.0005' and approach == 'uniform' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query4=f"llm=='GPT2' and dataset in ('adult', 'airline', 'texas') and LR=='0.0005' and approach == 'ood-using-adult' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query5=f"llm=='GPT2' and dataset in ('adult', 'airline', 'texas') and LR=='0.0005' and approach == 'ood-using-airline' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query6=f"llm=='GPT2' and dataset in ('adult', 'airline', 'texas') and LR=='0.0005' and approach == 'ood-using-texas' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"


index=["dataset", "llm", "finetune_type", "folder_name", "bs", "train_epoch", "chkt_epoch","eps", "clip",'fake_size', "LR", "shuffle", "impute", "wl", "approach"]
df_tmp=df.query(f"{query1} or {query2} or {query3} or {query4} or {query5} or {query6}")
df_tmp=df_tmp[df_tmp['exp'] != 'baselines_adult']
df_tmp=df_tmp[df_tmp['exp'] != 'baseline-test_airline']
df_tmp = df_tmp[df_tmp.fake_path.apply(lambda x: 'baselines' not in x)]
df_tmp['score']=df_tmp.apply(lambda x: x['score'] * 100 if x['metric'] == 'correlation_accuracy' or x['metric'] == 'pairwise_similarity' else x['score'], axis=1)

columns=["merged_metric_name2"]
tabular_metrics=['efficacy_f1', 'efficacy_auc', 'efficacy_accuracy', 'correlation_accuracy', 'pairwise_mean', 'histogram_mean']
# assert len(df_tmp.fake_path.unique()) == (6*2*4) + (5*1*4)

ROUND=1
df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score").fillna("-")
df_tmpp=df_tmpp.reset_index().set_index(['dataset', 'llm', 'approach'])[tabular_metrics].loc[['adult', 'airline', 'texas'], ['aim', 'mst', 'dpgan', 'dpctgan', 'dpvae', 'GPT2'], [-1, -1, -1, -1, -1, 'standard', 'uniform', 
                                                                                                                                                                                         'ood-using-adult', 
                                                                                                                                                                                         'ood-using-airline',
                                                                                                                                                                                         'ood-using-texas',]]
df_tmpp

/var/folders/z5/cs_w84312s1cxj7105940jzm0000gn/T/ipykernel_56685/1655157872.py:22: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score").fillna("-")


merged_metric_name2                  efficacy_f1   efficacy_auc  \
dataset llm     approach                                          
adult   aim     -1                  59.6$\pm$6.0   86.8$\pm$1.0   
        mst     -1                 39.6$\pm$19.0   76.8$\pm$1.0   
        dpgan   -1                 33.5$\pm$20.0   67.7$\pm$9.0   
        dpctgan -1                 42.2$\pm$20.0   78.0$\pm$7.0   
        dpvae   -1                   0.0$\pm$0.0   50.0$\pm$0.0   
        GPT2    standard           27.8$\pm$15.0   58.5$\pm$7.0   
                uniform            21.2$\pm$12.0   48.9$\pm$6.0   
                ood-using-airline  30.4$\pm$17.0   61.6$\pm$8.0   
                ood-using-texas    31.6$\pm$13.0   60.5$\pm$7.0   
airline aim     -1                  77.3$\pm$5.0   88.9$\pm$4.0   
        mst     -1                  72.2$\pm$6.0   83.4$\pm$5.0   
        dpgan   -1                 40.2$\pm$24.0  63.9$\pm$13.0   
        dpctgan -1                  67.1$\pm$8.0   76.7$\pm$8.0   
        dpvae   -1                 26.5$\pm$28.0  57.9$\pm$13.0   
        GPT2    standard            60.5$\pm$7.0   65.3$\pm$9.0   
                uniform             68.5$\pm$9.0  77.8$\pm$10.0   
                ood-using-adult    55.2$\pm$18.0  62.5$\pm$19.0   
                ood-using-texas    52.5$\pm$13.0  61.0$\pm$13.0   
texas   aim     -1                  84.5$\pm$1.0   98.3$\pm$0.0   
        mst     -1                  81.7$\pm$0.0   94.8$\pm$0.0   
        dpgan   -1                 13.7$\pm$16.0  58.3$\pm$12.0   
        dpctgan -1                 63.9$\pm$11.0   91.4$\pm$3.0   
        dpvae   -1                   0.0$\pm$0.0   50.0$\pm$0.0   
        GPT2    standard           55.4$\pm$10.0   90.2$\pm$5.0   
                uniform            23.5$\pm$15.0  59.6$\pm$15.0   
                ood-using-adult     74.8$\pm$4.0   96.7$\pm$1.0   
                ood-using-airline   74.3$\pm$5.0   96.4$\pm$0.0   

merged_metric_name2               efficacy_accuracy correlation_accuracy  \
dataset llm     approach                                                   
adult   aim     -1                     80.3$\pm$2.0         86.4$\pm$1.0   
        mst     -1                     72.8$\pm$2.0         70.0$\pm$1.0   
        dpgan   -1                    64.2$\pm$10.0         39.9$\pm$3.0   
        dpctgan -1                     75.7$\pm$3.0         51.3$\pm$3.0   
        dpvae   -1                     75.6$\pm$0.0         48.8$\pm$1.0   
        GPT2    standard               65.2$\pm$9.0         55.0$\pm$1.0   
                uniform               61.9$\pm$13.0         55.0$\pm$1.0   
                ood-using-airline      66.7$\pm$8.0         55.4$\pm$1.0   
                ood-using-texas        66.4$\pm$8.0         55.6$\pm$1.0   
airline aim     -1                     78.2$\pm$5.0         91.8$\pm$1.0   
        mst     -1                     75.2$\pm$4.0         72.7$\pm$0.0   
        dpgan   -1                     59.8$\pm$6.0         37.4$\pm$9.0   
        dpctgan -1                     68.0$\pm$6.0         31.7$\pm$2.0   
        dpvae   -1                     57.3$\pm$6.0         46.6$\pm$1.0   
        GPT2    standard               62.4$\pm$7.0         64.0$\pm$2.0   
                uniform                72.1$\pm$7.0         65.3$\pm$1.0   
                ood-using-adult       60.0$\pm$16.0         66.8$\pm$1.0   
                ood-using-texas       58.4$\pm$10.0         66.1$\pm$2.0   
texas   aim     -1                     94.2$\pm$1.0         81.0$\pm$2.0   
        mst     -1                     93.2$\pm$0.0         77.0$\pm$0.0   
        dpgan   -1                     78.3$\pm$8.0         36.1$\pm$7.0   
        dpctgan -1                    82.6$\pm$19.0         43.9$\pm$6.0   
        dpvae   -1                     82.5$\pm$0.0         62.1$\pm$1.0   
        GPT2    standard              77.1$\pm$11.0         70.3$\pm$1.0   
                uniform               67.3$\pm$17.0         68.2$\pm$0.0   
    

# Table 4

Numbers don't match exactly because the table below is averaged over all the experiments I ran but the reported table in the paper is based on single run (the first run) for DP-Standard and DP-2Stage-O, mostly because some of the experiments when shuffle is True did not run to completion.

In [7]:
eps=1

query1=f"llm=='GPT2'and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'standard' and impute=='False' and eps=={eps} and wl == '-1'"
query2=f"llm=='GPT2'and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'standard' and shuffle == 'True' and impute=='True' and eps=={eps} and wl == '-1'"
query3=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and shuffle == 'True' and impute=='False' and eps=='nondp'"
query4=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and shuffle == 'False' and impute=='False' and eps=='nondp'"
query5=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'uniform' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query6=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'uniform' and shuffle == 'True' and impute=='True' and eps=={eps} and wl == '0.65'"
query7=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-adult' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query8=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-adult' and shuffle == 'True' and impute=='True' and eps=={eps} and wl == '0.65'"
query9=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-airline' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query10=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-airline' and shuffle == 'True' and impute=='True' and eps=={eps} and wl == '0.65'"

index=["dataset", "llm", "finetune_type", "folder_name", "bs", "train_epoch", "chkt_epoch","eps", "clip",'fake_size', "LR", "shuffle", "impute", "wl", "approach"]
df_tmp=df.query(f"{query1} or {query2} or {query3} or {query4} or {query5} or {query6} or {query7} or {query8} or {query9} or {query10}")
df_tmp=df_tmp[df_tmp['exp'] != 'baselines_adult']
df_tmp=df_tmp[df_tmp['exp'] != 'baseline-test_airline']
df_tmp = df_tmp[df_tmp.fake_path.apply(lambda x: 'baselines' not in x)]
df_tmp['score']=df_tmp.apply(lambda x: x['score'] * 100 if x['metric'] == 'correlation_accuracy' or x['metric'] == 'pairwise_similarity' else x['score'], axis=1)

columns=["merged_metric_name2"]
tabular_metrics=['efficacy_f1', 'efficacy_auc', 'efficacy_accuracy', 'histogram_mean']
# assert len(df_tmp.fake_path.unique()) == (6*2*4) + (5*1*4)

ROUND=1
df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score").fillna("-")
df_tmpp=df_tmpp.reset_index().set_index(['dataset', 'llm', 'eps', 'approach', 'shuffle'])[tabular_metrics].loc[['adult', 'airline'], :, ['nondp', 1], ['standard', 'uniform', 'ood-using-adult', 'ood-using-airline'], ['True', 'False']]

df_tmpp

/var/folders/z5/cs_w84312s1cxj7105940jzm0000gn/T/ipykernel_56685/4142321769.py:26: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score").fillna("-")


merged_metric_name2                             efficacy_f1   efficacy_auc  \
dataset llm  eps   approach          shuffle                                 
adult   GPT2 nondp standard          True      68.9$\pm$0.0   90.7$\pm$0.0   
                                     False     69.0$\pm$1.0   90.7$\pm$0.0   
             1     standard          True     19.5$\pm$20.0  54.2$\pm$12.0   
                                     False    27.8$\pm$15.0   58.5$\pm$7.0   
                   uniform           True     19.7$\pm$13.0   48.0$\pm$8.0   
                                     False    21.2$\pm$12.0   48.9$\pm$6.0   
                   ood-using-airline True     20.5$\pm$19.0   53.8$\pm$9.0   
                                     False    30.4$\pm$17.0   61.6$\pm$8.0   
airline GPT2 nondp standard          True      89.6$\pm$5.0   95.9$\pm$3.0   
                                     False    76.5$\pm$24.0   91.3$\pm$7.0   
             1     standard          True     43.0$\pm$28.0  68.5$\pm$13.0   
                                     False     60.5$\pm$7.0   65.3$\pm$9.0   
                   uniform           True     48.8$\pm$11.0   57.5$\pm$7.0   
                                     False     68.5$\pm$9.0  77.8$\pm$10.0   
                   ood-using-adult   True     55.5$\pm$15.0   69.8$\pm$7.0   
                                     False    55.2$\pm$18.0  62.5$\pm$19.0   

merged_metric_name2                          efficacy_accuracy histogram_mean  
dataset llm  eps   approach          shuffle                                   
adult   GPT2 nondp standard          True         83.7$\pm$2.0   90.7$\pm$1.0  
                                     False        83.3$\pm$2.0   90.2$\pm$0.0  
             1     standard          True        64.0$\pm$13.0   87.7$\pm$1.0  
                                     False        65.2$\pm$9.0   85.7$\pm$2.0  
                   uniform           True        61.9$\pm$13.0   87.6$\pm$1.0  
                                     False       61.9$\pm$13.0   86.7$\pm$1.0  
                   ood-using-airline True        65.1$\pm$12.0   87.4$\pm$1.0  
                                     False        66.7$\pm$8.0   88.5$\pm$1.0  
airline GPT2 nondp standard          True         91.4$\pm$4.0   90.8$\pm$1.0  
                                     False       84.0$\pm$10.0   93.5$\pm$3.0  
             1     standard          True        64.1$\pm$10.0   93.4$\pm$1.0  
                                     False        62.4$\pm$7.0   90.3$\pm$3.0  
                   uniform           True         55.2$\pm$5.0   93.4$\pm$1.0  
                                     False        72.1$\pm$7.0   90.7$\pm$1.0  
                   ood-using-adult   True         65.9$\pm$7.0   92.5$\pm$1.0  
                                     False       60.0$\pm$16.0   92.5$\pm$1.0

In [8]:
df2 = pd.read_csv("./results/perp_metrics_synth_data_paths_29.09.24.csv")
df2["fake_path"]=df2.name
df2.fake_path = df2.fake_path.apply(lambda x: x.replace("/workspace", "/home/c01teaf/CISPA-az6/llm_tg-2024/llm-tg"))
df2[["dataset", "setting", "llm", "finetune_type"]] = df2.apply(lambda x: get_dslf(x), axis=1, result_type="expand")
df2["unique_name"] = df2.apply(lambda x: "_".join(x['fake_path'].split("/")[1:-2]),axis=1)

df2['bs']=df2.fake_path.apply(lambda x: get_bs(x))
df2['exp']=df2.fake_path.apply(lambda x: get_exp(x))
df2['folder_name']=df2.fake_path.apply(lambda x: get_folder_name(x))
df2['train_epoch']=df2.fake_path.apply(lambda x: get_train_ep(x))

df2['chkt_epoch'] = df2.fake_path.apply(lambda x: get_epoch(x))
df2['eps'] = df2.fake_path.apply(lambda x: get_eps(x))
df2['clip']=df2.fake_path.apply(lambda x: get_clip(x))
df2=df2.fillna("None")

df2['LR']=df2.exp.apply(lambda x: x.split("-")[0].strip("LR") if "LR" in x else -1)
df2['shuffle']=df2.exp.apply(lambda x: x.split("_")[3].split("-")[-1] if "shuffle" in x else "-1")
df2['impute']=df2.folder_name.apply(lambda x: x.split("_")[-1].split("-")[-1] if "impute" in x else -1)
df2['wl']=df2.exp.apply(lambda x: x.split("_")[-1].strip("wl") if "wl" in x else "-1")
index2=index+["LR", "shuffle", "impute","wl"]

def f(x):
    if "ood" in x:
        return "OOD"
    elif "uniform" in x:
        return "uniform"
    elif "Standard" in x:
        return "standard"
    else:
        return -1
    
df2['approach']=df2.exp.apply(lambda x: f(x))
index2=index2+["approach"]

In [9]:
query1=f"dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'standard' and shuffle == 'True' and eps==1 and train_epoch == 10 and wl=='-1'"
query2=f"dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'standard' and shuffle == 'False' and eps==1 and train_epoch == 10 and wl=='-1'"
query3=f"dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'uniform' and shuffle == 'False' and eps==1 and train_epoch == 10 and wl=='0.65'"
query4=f"dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'uniform' and shuffle == 'True' and eps==1 and train_epoch == 10 and wl=='0.65'"
query5=f"dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'OOD' and shuffle == 'True' and eps==1 and train_epoch == 10 and wl=='0.65'"
query6=f"dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'OOD' and shuffle == 'False' and eps==1 and train_epoch == 10 and wl=='0.65'"
query7=f"dataset in ('adult', 'airline') and LR=='0.0005' and shuffle == 'False' and eps=='nondp' and train_epoch == 10 and wl=='-1'"
query8=f"dataset in ('adult', 'airline') and LR=='0.0005' and shuffle == 'True' and eps=='nondp' and train_epoch == 10 and wl=='-1'"


index2=["dataset", "llm", "finetune_type", "folder_name", "bs", "train_epoch", "chkt_epoch","eps", "clip"]

index2=index2+["LR", "shuffle", "impute","wl"]
index2=index2+["approach"]

df_tmp=df2.query(f"{query1} or {query2} or {query3} or {query4} or {query5} or {query6} or {query7} or {query8}")
columns2=['value perp']
ROUND2=3
df_tmpp=df_tmp.groupby(index2).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND2)), str(x.std().round(ROUND-1))]))[columns2]
df_tmpp.reset_index().set_index(['dataset', 'llm', 'eps', 'approach', 'shuffle'])[['value perp']].loc[['adult', 'airline'], :, ['nondp', 1], ['standard', 'uniform', 'OOD'], ['True', 'False']]

/var/folders/z5/cs_w84312s1cxj7105940jzm0000gn/T/ipykernel_56685/1660854799.py:19: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df_tmpp=df_tmp.groupby(index2).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND2)), str(x.std().round(ROUND-1))]))[columns2]


value perp
dataset llm  eps   approach shuffle               
adult   GPT2 nondp standard True     2.398$\pm$0.0
                            False    2.482$\pm$0.0
             1     standard True     3.514$\pm$0.0
                            False    3.216$\pm$0.0
                   uniform  True     2.973$\pm$0.0
                            False    2.887$\pm$0.0
                   OOD      True     3.269$\pm$0.0
                            False    3.058$\pm$0.0
airline GPT2 nondp standard True     2.865$\pm$0.0
                            False    2.901$\pm$0.0
             1     standard True     4.297$\pm$0.0
                            False    3.798$\pm$0.0
                   uniform  True     3.932$\pm$0.0
                            False    3.633$\pm$0.0
                   OOD      True     3.964$\pm$0.0
                            False     3.74$\pm$0.0

# Table 5

In [10]:
eps=1

query1=f"llm=='GPT2'and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'standard' and impute=='False' and eps=={eps} and wl == '0.65'"
query2=f"llm=='GPT2'and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'standard' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '-1'"
query3=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'uniform' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query4=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'uniform' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '-1'"
query5=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-adult' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query6=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-adult' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '-1'"
query7=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-airline' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '0.65'"
query8=f"llm=='GPT2' and dataset in ('adult', 'airline') and LR=='0.0005' and approach == 'ood-using-airline' and shuffle == 'False' and impute=='False' and eps=={eps} and wl == '-1'"

index=["dataset", "llm", "finetune_type", "folder_name", "bs", "train_epoch", "chkt_epoch","eps", "clip",'fake_size', "LR", "shuffle", "impute", "wl", "approach"]
df_tmp=df.query(f"{query1} or {query2} or {query3} or {query4} or {query5} or {query6} or {query7} or {query8}")
df_tmp=df_tmp[df_tmp['exp'] != 'baselines_adult']
df_tmp=df_tmp[df_tmp['exp'] != 'baseline-test_airline']
df_tmp = df_tmp[df_tmp.fake_path.apply(lambda x: 'baselines' not in x)]
df_tmp['score']=df_tmp.apply(lambda x: x['score'] * 100 if x['metric'] == 'correlation_accuracy' or x['metric'] == 'pairwise_similarity' else x['score'], axis=1)

columns=["merged_metric_name2"]
tabular_metrics=['efficacy_f1', 'efficacy_auc', 'efficacy_accuracy', 'histogram_mean']
# assert len(df_tmp.fake_path.unique()) == (6*2*4) + (5*1*4)

ROUND=1
df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score").fillna("-")
df_tmpp=df_tmpp.reset_index().set_index(['dataset', 'llm', 'eps', 'approach', 'wl'])[tabular_metrics].loc[['adult', 'airline'], :, ['nondp', 1], ['standard', 'uniform', 'ood-using-adult', 'ood-using-airline'], ['-1', '0.65']]

df_tmpp

/var/folders/z5/cs_w84312s1cxj7105940jzm0000gn/T/ipykernel_56685/615234026.py:24: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df_tmpp = df_tmp.groupby(index+columns).agg(lambda x:"$\pm$".join([str(x.mean().round(ROUND)), str(x.std().round(ROUND-1))]))[["score"]].reset_index().pivot(columns=columns, index=index, values="score").fillna("-")
/var/folders/z5/cs_w84312s1cxj7105940jzm0000gn/T/ipykernel_56685/615234026.py:25: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  df_tmpp=df_tmpp.reset_index().set_index(['dataset', 'llm', 'eps', 'approach', 'wl']

merged_metric_name2                        efficacy_f1   efficacy_auc  \
dataset llm  eps approach          wl                                   
adult   GPT2 1   standard          -1    27.8$\pm$15.0   58.5$\pm$7.0   
                                   0.65  28.9$\pm$17.0   60.9$\pm$8.0   
                 uniform           -1    20.8$\pm$14.0   49.7$\pm$7.0   
                                   0.65  21.2$\pm$12.0   48.9$\pm$6.0   
                 ood-using-airline -1    30.3$\pm$15.0   60.4$\pm$7.0   
                                   0.65  30.4$\pm$17.0   61.6$\pm$8.0   
airline GPT2 1   standard          -1     60.5$\pm$7.0   65.3$\pm$9.0   
                                   0.65   59.9$\pm$7.0   65.7$\pm$9.0   
                 uniform           -1    65.0$\pm$10.0   74.5$\pm$9.0   
                                   0.65   68.5$\pm$9.0  77.8$\pm$10.0   
                 ood-using-adult   -1    48.8$\pm$17.0  55.6$\pm$19.0   
                                   0.65  55.2$\pm$18.0  62.5$\pm$19.0   

merged_metric_name2                     efficacy_accuracy histogram_mean  
dataset llm  eps approach          wl                                     
adult   GPT2 1   standard          -1        65.2$\pm$9.0   85.7$\pm$2.0  
                                   0.65      66.8$\pm$8.0   87.4$\pm$2.0  
                 uniform           -1       63.0$\pm$12.0   87.3$\pm$1.0  
                                   0.65     61.9$\pm$13.0   86.7$\pm$1.0  
                 ood-using-airline -1        65.8$\pm$9.0   87.3$\pm$1.0  
                                   0.65      66.7$\pm$8.0   88.5$\pm$1.0  
airline GPT2 1   standard          -1        62.4$\pm$7.0   90.3$\pm$3.0  
                                   0.65      62.9$\pm$7.0   91.6$\pm$4.0  
                 uniform           -1        69.3$\pm$6.0   91.1$\pm$1.0  
                                   0.65      72.1$\pm$7.0   90.7$\pm$1.0  
                 ood-using-adult   -1       54.4$\pm$15.0   92.4$\pm$1.0  
                                   0.65     60.0$\pm$16.0   92.5$\pm$1.0